In [ ]:
#Hello!

In [ ]:
#Rotten Tomatoes Movies in Theaters + total page num = 5, thus all in-theater movie data By Sherry
import pandas as pd
import requests
from bs4 import BeautifulSoup

def get_in_theater(page_num):
  movie_dict = {'Title':[], 'Release Date': [], 'Critics Rating':[], 'Audience Rating': []}
  url_load = "https://www.rottentomatoes.com/browse/movies_in_theaters/sort:popular?page="+page_num
  resp = requests.get(url_load)
  soup = BeautifulSoup(resp.text, 'html.parser')

  for tag in soup.find_all('a', {'data-track': 'scores'}):
     name = tag.find('span', {'data-qa': 'discovery-media-list-item-title'}).text
     date = tag.find('span', {'data-qa': 'discovery-media-list-item-start-date'}).text
     audience_score = tag.find('score-pairs-deprecated')['audiencescore']
     critic_score = tag.find('score-pairs-deprecated')['criticsscore']
     movie_dict['Title'].append(name.strip())
     movie_dict['Release Date'].append(date.strip())
     movie_dict['Critics Rating'].append(critic_score.strip())
     movie_dict['Audience Rating'].append(audience_score.strip())

  return pd.DataFrame(movie_dict)

df1 = get_in_theater('10')
styled_df1 = df1.style.set_properties(subset=df1.columns[0],**{'text-align': 'left'})
display(styled_df1)

In [ ]:
 #Top 8 in-theater movies
df1.head(8).style.set_properties(subset=df1.columns[0],**{'text-align': 'left'})

In [ ]:
#Top 8 in-theater movies saved in a list
import re

title_10 = df1.iloc[:8, 0].tolist()
url_title = []
for title in title_10:
  movie_title = title
  movie_title_fixed = re.sub(r'[ ,:]', '_', movie_title)
  movie_title_final = re.sub(r'_{2,}', '_', movie_title_fixed).lower()
  url_title.append(movie_title_final)

print(url_title)

['dune_part_two', 'kung_fu_panda_4', 'damsel', 'poor_things', 'imaginary', 'cabrini', 'the_zone_of_interest', 'madame_web']


In [ ]:
#Top 8 in-theater movies with descriptions
import pandas as pd
import requests

def get_movie_des(title):
  des_dict = {'Title':[], 'Description':[], 'Genre': [], 'Rating':[]}
  url_des = "https://www.rottentomatoes.com/m/"+title
  resp = requests.get(url_des)
  soup = BeautifulSoup(resp.text, 'html.parser')

#<p slot="info" class="info" data-qa="score-panel-subtitle">2024, Kids &amp; family/Comedy, 1h 34m</p>
 # for tag in soup.find_all('ul', {'id': 'info'}):
  for tag in soup.find_all('div', {'id': 'main'}):
     name = tag.find('h1', {'data-qa': 'score-panel-title'}).text
     des = tag.find('p', {'data-qa': 'score-panel-subtitle'}).text
     genre = tag.find('span', {'class': 'genre'}, {'data-qa': 'movie-info-item-value'}).text
     #runtime = tag.find('span', {'data-qa': 'movie-info-item-value'}, 'time').text
     rating = tag.find('span', {'data-qa': 'movie-info-item-value'}).text

     des_dict['Title'].append(name.strip())
     des_dict['Description'].append(des.strip())
     des_dict['Genre'].append(genre.strip())
     #description['Runtime'].append(runtime.strip())
     des_dict['Rating'].append(rating.strip())
     #description['Comment'].append(audience_score.strip())

  return pd.DataFrame(des_dict)

dfs = []

for title in url_title:
  df2 = pd.DataFrame(get_movie_des(title))
  dfs.append(df2)

result_df = pd.concat(dfs, axis=0)
result_df.reset_index(drop=True, inplace=True)
styled_dfs = result_df.style.set_properties(subset=result_df.columns[0],**{'text-align': 'left'})
display(styled_dfs)

In [ ]:
#A copy of the previous one and replace description with runtime
import pandas as pd
import requests

def get_movie_des(title):
  des_dict = {'Title':[], 'Runtime': [], 'Genre': [], 'Rating':[]}
  url_des = "https://www.rottentomatoes.com/m/"+title
  resp = requests.get(url_des)
  soup = BeautifulSoup(resp.text, 'html.parser')

#<p slot="info" class="info" data-qa="score-panel-subtitle">2024, Kids &amp; family/Comedy, 1h 34m</p>
 # for tag in soup.find_all('ul', {'id': 'info'}):
  for tag in soup.find_all('div', {'id': 'main'}):
     name = tag.find('h1', {'data-qa': 'score-panel-title'}).text
     des = tag.find('p', {'data-qa': 'score-panel-subtitle'}).text
     genre = tag.find('span', {'class': 'genre'}, {'data-qa': 'movie-info-item-value'}).text
     year = des.split(', ')[0]
     runtime = des.split(', ')[2]
     rating = tag.find('span', {'data-qa': 'movie-info-item-value'}).text

     des_dict['Title'].append(name.strip())
     #des_dict['Description'].append(des.strip())
     des_dict['Runtime'].append(runtime.strip())
     des_dict['Genre'].append(genre.strip())
     des_dict['Rating'].append(rating.strip())
     #description['Comment'].append(audience_score.strip())

  return pd.DataFrame(des_dict)

dfs = []

for title in url_title:
  df2 = pd.DataFrame(get_movie_des(title))
  dfs.append(df2)

result_df = pd.concat(dfs, axis=0)
result_df.reset_index(drop=True, inplace=True)
styled_dfs = result_df.style.set_properties(subset=result_df.columns[0],**{'text-align': 'left'})
display(styled_dfs)

In [ ]:
combined_df = pd.merge(df1.head(8), result_df, on="Title", how="inner")

# Reset index
combined_df.reset_index(drop=True, inplace=True)

# Display the combined DataFrame
styled_combined_df = combined_df.style.set_properties(subset=combined_df.columns[0], **{'text-align': 'left'})
display(styled_combined_df)

#One problem: If the newest movie shares the same name as an old movie, the description here will fetch the old one. I'm not sure how to solve this problem. i.e. Damsel